next step:  
1. 在开头进行完整的数据清洗和补全，保证高质量的后续数据使用
2. 统一图像风格和数据调用
3. 保存输出结果，增强统计学习和研究可信度

| 阶段      | 分析模块      | 分析变量                            | 分析目标               | 关键统计指标               | 可视化方法            |
| :------ | :-------- | :------------------------------ | :----------------- | :------------------- | :--------------- |
| **阶段1** | 数据加载与质量检查 | `paper_info`, `paper_citations` | 了解数据结构、识别缺失值与异常情况  | 数据维度、字段类型、缺失值数量      | 无需可视化，输出info()摘要 |
| **阶段2** | 论文属性分析    | `year` (发表年份)                   | 识别时间跨度、出版高峰期、数据完整性 | 年份范围、中位数、年度论文数       | 直方图、年度趋势折线图      |
|         |           | `journal` (期刊)                  | 识别核心期刊、评估期刊集中度     | 期刊总数、Top5占比、频数分布     | 条形图(Top15)       |
|         |           | `author` (作者)                   | 识别高产作者、检验是否符合幂律分布  | 作者总数、平均/中位数发文量、最高发文量 | 对数坐标直方图、箱线图      |
|         |           | `subfield` (子领域)                | 了解领域覆盖广度、检查类别不平衡   | 类别数量、各类别占比           | 条形图/饼图           |
| **阶段3** | 引用关系分析    | `indegree` (被引次数)               | 评估学术影响力分布、识别高被引论文  | 均值、中位数、零引用比例         | 双对数直方图、箱线图       |
|         |           | `outdegree` (引用他人次数)            | 了解论文参考文献数量、识别综述型论文 | 均值、中位数、最大值           | 直方图+KDE曲线        |
|         |           | `time_lag` (引用时间间隔)             | 理解知识传播速度、引用时效性     | 平均时间差、分布形态           | 直方图+KDE曲线        |
| **阶段4** | 综合洞察报告    | 全部变量                            | 生成关键洞察摘要、指导后续分析    | 数据规模、核心指标汇总          | 无需可视化，文本报告       |


| 变量类型     | 必读指标       | 关键洞察方向    | 后续分析衔接   |
| :------- | :--------- | :-------- | :------- |
| **时间变量** | 年份范围、增长趋势  | 网络演化阶段划分  | 时间切片网络构建 |
| **类别变量** | 期刊/领域集中度   | 领域异质性评估   | 跨领域链接预测  |
| **幂律变量** | 作者/被引的α指数  | 网络无标度特性验证 | 节点重要性采样  |
| **关系变量** | 零引用比例、时间延迟 | 数据质量与偏误识别 | 负样本构造策略  |



In [ ]:
# 7. Abstract Length Distribution
# Analyze the length of abstracts
paper_info['abstract_length'] = paper_info['abstract'].fillna('').apply(lambda x: len(str(x).split()))

plt.figure(figsize=(10, 6))
sns.histplot(paper_info['abstract_length'], bins=30, kde=True, color='green')
plt.title('Distribution of Abstract Lengths (Word Count)')
plt.xlabel('Number of Words')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# 8. Citation Network Graph (Top 50)
# Visualize the citation network for the top 50 most cited papers
import networkx as nx

# Filter for top 50 cited papers to keep the graph readable
top_50_ids = citation_counts.head(50)['paper_id'].tolist()
subset_edges = paper_citations[paper_citations['target'].isin(top_50_ids) & paper_citations['source'].isin(top_50_ids)]

G = nx.from_pandas_edgelist(subset_edges, source='source', target='target', create_using=nx.DiGraph())

plt.figure(figsize=(12, 12))
# Use spring layout
pos = nx.spring_layout(G, k=0.5, seed=42)
nx.draw_networkx_nodes(G, pos, node_size=100, node_color='skyblue')
nx.draw_networkx_edges(G, pos, alpha=0.3, arrows=True)
# Optional: Add labels if needed, but might be cluttered
# nx.draw_networkx_labels(G, pos, font_size=8)
plt.title('Citation Network of Top 50 Cited Papers')
plt.axis('off')
plt.show()

我们可以从期刊分布、引用关系、文本内容等多个维度进行可视化分析。

以下是为您设计的 8 个研究问题 以及对应的 可视化方案：

1. 各期刊的论文数量分布是怎样的？
研究目的: 了解数据集中四大统计学期刊的收录比例，看是否存在某些期刊的论文数远多于其他期刊。
数据处理: 对 paper_information 表中的 publisher 列进行计数统计。
可视化方式: 柱状图 (Bar Chart) 或 饼图 (Pie Chart)。
X轴: 期刊名称
Y轴: 论文数量
2. 哪些论文是“核心文献”（被引用次数最多）？
研究目的: 识别在该领域内影响力最大的论文。
数据处理: 利用 paper_edge_citation 表，计算每个 target (被引论文ID) 的出现次数（即入度 In-degree），取前 10-20 名。
可视化方式: 水平柱状图 (Horizontal Bar Chart)。
Y轴: 论文标题 (需要关联 paper_information 获取标题)
X轴: 被引用次数
3. 论文被引用次数的分布规律是什么？
研究目的: 验证是否存在“长尾效应”或“二八定律”（即少数论文拥有绝大多数引用）。
数据处理: 计算每篇论文的被引用次数，统计每个引用次数对应的论文数量。
可视化方式: 直方图 (Histogram) 或 双对数坐标图 (Log-Log Plot)。
X轴: 被引用次数
Y轴: 拥有该引用次数的论文数量（频数）
4. 四大期刊之间的相互引用关系如何？（学术圈层分析）
研究目的: 观察期刊之间是否存在“引用壁垒”或“偏好”，例如 JASA 是否更倾向于引用 JASA 自己的文章。
数据处理: 将 paper_edge_citation 中的 source 和 target 分别映射到对应的 publisher (期刊名)，构建一个 4x4 的引用矩阵。
可视化方式: 热力图 (Heatmap) 或 桑基图 (Sankey Diagram)。
行: 引用方期刊 (Source Journal)
列: 被引方期刊 (Target Journal)
颜色深浅: 引用次数
5. 当前统计学领域最热门的研究关键词是什么？
研究目的: 挖掘该领域的研究热点。
数据处理: 对 paper_information 中的 keywords 列进行分割（通常以逗号分隔）和清洗，统计词频。
可视化方式: 词云图 (Word Cloud) 或 Top 20 关键词条形图。
词云: 词越大代表出现频率越高。
6. 论文的参考文献数量与被引用次数是否存在相关性？
研究目的: 探究“引经据典”更多的论文是否更容易被他人引用。
数据处理:
计算每篇论文的参考文献数量（对 references 列按分隔符 :: 分割后计数）。
计算每篇论文的被引用次数。
可视化方式: 散点图 (Scatter Plot)。
X轴: 参考文献数量
Y轴: 被引用次数
辅助: 可以添加一条回归趋势线。
7. 论文摘要 (Abstract) 的长度分布如何？
研究目的: 了解统计学论文摘要的普遍篇幅。
数据处理: 计算 abstract 列的字符长度或单词数量。
可视化方式: 直方图 (Histogram) 配合 核密度估计曲线 (KDE)。
X轴: 摘要单词数
Y轴: 频率
8. 核心引文网络结构是怎样的？
研究目的: 直观展示论文之间的引用拓扑结构，发现聚类（Community）。
数据处理: 由于 2万多条边太密集，建议只筛选 被引用次数 Top 50 或 Top 100 的论文及其之间的引用关系。
可视化方式: 网络关系图 (Network Graph) (使用 networkx 库)。
节点: 论文
连线: 引用关系
节点大小: 被引用次数
节点颜色: 所属期刊

In [ ]:
# Data Preparation: Extract Year
# 时序分析

import re

def extract_year(row):
    doi = str(row['doi'])
    # AOS: 10.1214/13-AOS1143 -> 2013
    match = re.search(r'10\.1214/(\d{2})-AOS', doi)
    if match:
        return 2000 + int(match.group(1))
    
    # JASA: 10.1198/jasa.2011.tm11321 -> 2011
    match = re.search(r'10\.1198/jasa\.(\d{4})', doi)
    if match:
        return int(match.group(1))
    
    # JRSSB: 10.1111/j.1467-9868.2007.00605.x -> 2007
    match = re.search(r'10\.1111/j\.1467-9868\.(\d{4})', doi)
    if match:
        return int(match.group(1))
    
    # Fallback: Extract max year from references
    refs = str(row['references'])
    if pd.isna(refs):
        return np.nan
    
    # Find all 4-digit years in references (19xx or 20xx)
    years = re.findall(r'(?:19|20)\d{2}', refs)
    if years:
        # The publication year is likely >= the latest reference year
        # We use the max reference year as a proxy
        return int(max(years))
    
    return np.nan

paper_info['year'] = paper_info.apply(extract_year, axis=1)

# Filter out invalid years (e.g. future years or too old if any)
paper_info = paper_info[(paper_info['year'] >= 1900) & (paper_info['year'] <= 2025)]

print(f"Extracted years for {paper_info['year'].notna().sum()} papers.")
paper_info['year'].value_counts().sort_index().plot(kind='bar', figsize=(15, 5), title='Paper Distribution by Year (Extracted)')
plt.show()

我已经为您添加了 4 个新的图表代码，并包含了一个必要的数据预处理步骤（提取年份）。

**新增内容：**

1.  **数据预处理 (Data Preparation)**:
    *   由于原始数据中没有直接的“年份”列，我编写了一段代码从 `doi` (数字对象唯一标识符) 和 `references` (参考文献) 中提取年份。这是进行时间序列分析的基础。

2.  **期刊引用集中度随年份变化 (Gini Coefficient)**:
    *   计算每年的引用基尼系数。基尼系数越高，说明引用越集中在少数几篇论文上（“赢家通吃”现象）。

3.  **期刊发表量随年份变化 (Stacked Histogram)**:
    *   展示四大期刊每年的发文量趋势，可以直观看到各期刊的活跃度变化。

4.  **QQ-Plot (正态分布检验)**:
    *   使用 `log(citations + 1)` 进行 QQ 图绘制。因为引用数据通常呈现幂律分布（长尾），直接画 QQ 图会严重偏离，取对数后可以更好地观察其是否符合对数正态分布。

5.  **Box Plot (箱线图)**:
    *   展示各期刊引用次数的分布情况，特别是**离群值 (Outliers)**（即那些引用量极高的“神作”）。为了显示清晰，使用了对数坐标轴。

In [ ]:
# 10. Journal Publication Volume over Time
# Visualize the number of papers published by each journal over the years
plt.figure(figsize=(14, 7))
# Fix: Use sns.move_legend instead of plt.legend to avoid UserWarning
ax = sns.histplot(data=paper_info, x='year', hue='publisher', multiple='stack', binwidth=1, palette='viridis')
plt.title('Journal Publication Volume over Time')
plt.xlabel('Year')
plt.ylabel('Number of Papers')
try:
    sns.move_legend(ax, "upper left", bbox_to_anchor=(1.05, 1), title='Journal')
except:
    # Fallback for older seaborn versions
    plt.legend(title='Journal', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# Missing Data Analysis & Extraction
# Objective: Analyze and extract rows with missing values (excluding 'year')
# Note: Treating 'NO DOI' and 'no abtract' as missing values.

# 1. Define columns to check (excluding 'year' as requested)
columns_to_check = [col for col in paper_info.columns if col not in ['year', 'temp_year']]

# 2. Define a custom missing check function
def is_missing(series):
    if series.name == 'doi':
        return series.isnull() | (series == 'NO DOI')
    if series.name == 'abstract':
        # Check for "no abtract" (user specified) and common variations
        return series.isnull() | series.isin(['no abtract', 'no abstract', 'No Abstract'])
    return series.isnull()

# Apply check to create a boolean mask
missing_mask = paper_info[columns_to_check].apply(is_missing)

# 3. Calculate missing statistics
missing_counts = missing_mask.sum()
missing_percentages = (missing_counts / len(paper_info)) * 100

# Filter only columns with missing values
missing_stats = pd.DataFrame({
    'Missing Count': missing_counts,
    'Percentage': missing_percentages
})
missing_stats = missing_stats[missing_stats['Missing Count'] > 0].sort_values('Percentage', ascending=False)

print("=== Missing Data Statistics (excluding 'year') ===")
print(missing_stats)

if not missing_stats.empty:
    # 4. Visualization 1: Bar Chart of Missing Percentages
    plt.figure(figsize=(10, 5))
    # Fix: Assign hue to x variable and set legend=False to avoid FutureWarning
    ax = sns.barplot(x=missing_stats.index, y='Percentage', data=missing_stats, hue=missing_stats.index, legend=False, palette='Reds_r')
    plt.title('Percentage of Missing Values by Column (incl. placeholders)')
    plt.ylabel('Missing Percentage (%)')
    plt.xlabel('Columns')
    plt.xticks(rotation=45)
    
    # Add labels
    for i, v in enumerate(missing_stats['Percentage']):
        ax.text(i, v + 0.1, f"{v:.2f}%", ha='center', va='bottom')
    plt.show()

    # 5. Visualization 2: Heatmap of Missingness
    plt.figure(figsize=(12, 6))
    sns.heatmap(missing_mask, cbar=False, cmap='viridis', yticklabels=False)
    plt.title('Missing Values Matrix (Yellow represents missing data)')
    plt.xlabel('Columns')
    plt.ylabel('Row Index')
    plt.show()

    # 6. Extract rows with missing values into a new DataFrame
    # We select rows where ANY of the checked columns are considered missing
    missing_entries_df = paper_info[missing_mask.any(axis=1)].copy()
    
    print(f"\n=== Extraction Result ===")
    print(f"Created 'missing_entries_df' with {len(missing_entries_df)} rows.")
    print("Preview of missing entries:")
    print(missing_entries_df.head())
    
    # Optional: Save to CSV for inspection
    # missing_entries_df.to_csv('missing_data_entries.csv', index=False)
else:
    print("\nGreat! No missing values found in the checked columns.")